In [1]:
## Comprehensive list of useful tools

import matplotlib.patches as patches

In [ ]:
# These values change depending on the run used
pt = np.array([0, 5, 350])
xl = np.array([0,-10,-200])
xh = xl + np.array([1600,20,400])
gdims = np.array([10240,1,2560])
dx = (xh - xl) / gdims

In [ ]:
# Interp function for Get_fields fuction 
def interp(a, pt):
    n = ((pt - xl)/dx).astype(int)
    return a[n[0],n[1],n[2]]

In [ ]:
# Retrieves E and B at a given 3D position r
# Requires interp function above
def Get_Fields(r):
    Ex, Ey, Ez = fp['ex_ec'], fp['ey_ec'], fp['ez_ec']
    Bx, By, Bz = fp['hx_fc'], fp['hy_fc'], fp['hz_fc']
    
    
    r = np.array([r[0],0,r[2]]) # This Resets the y position to zero
    Ex = interp(Ex,r)
    Ey = interp(Ey,r)
    Ez = interp(Ez,r)
    Bx = interp(Bx,r)
    By = interp(By,r)
    Bz = interp(Bz,r)
    
    E = np.array([Ex, Ey, Ez])
    B = np.array([Bx, By, Bz])
    
    return E, B

In [ ]:
# Boris particle push
def Push(r0,p0,n):
    #Define position (r) & momentum (p) 
    r = np.zeros((n+1,3))
    p = np.zeros((n+1,3))
    
    #Initial r & p
    r[0] = r0
    p[0] = p0
    
    for i in range(n):

            E, B = Get_Fields(r[i])

            # Convenient rotation vector 
            b_rot = q*B*dt/2*m

            # 1st half of electric push
            p_1 = p[i] + q*E*dt/2*m

            # Rotation by VxB
            p_2 = p_1 + 2*VxB((p_1 + VxB(p_1,b_rot))/(1+np.linalg.norm(b_rot)**2),b_rot)

            # 2nd half of electric push 
            p[i+1] = p_2 + q*E*dt/2*m
            r[i+1] = r[i] + p[i+1]*dt
    
    return r,p


In [ ]:
# Code meant to select from specific pieces of a distribution function 
# Assumes there is a defined pandas dataframe (df) of particles



# Arc Selection
# Uses two concentric circles of radii rs and rl and selects particles that fall between those two radii on a distribution function

rs = 0.7  # Radius of small circle
rl = 1    # Radius of large circle
df['r'] = (df['px']**2 + df['py']**2)**0.5 # Defining a new pandas variable for radius (in this case, r is in x-y momentum space)   
df = df.loc[lambda df: df['r'] < rl]
df = df.loc[lambda df: df['r'] > rs]
    

    
    
# Square Selection  ((HAS NOT BEEN TESTED))
# Limits particles considered to those that fall within the bounds of the set x,y,z limits (again, in momentum space here)
# This is done on top of the Arc selection, so for example, with both of these a semicircular piece of the df can be chosen

x_range = [-1,1]
y_range = [-1,1]
z_range = [-1,1]

df = df.loc[lambda df: df['px'] > x_range[0]]
df = df.loc[lambda df: df['px'] < x_range[1]]
df = df.loc[lambda df: df['py'] > y_range[0]]
df = df.loc[lambda df: df['py'] < y_range[1]]
df = df.loc[lambda df: df['pz'] > z_range[0]]
df = df.loc[lambda df: df['pz'] < z_range[1]]




kwargs = {"gridsize": 80, "extent": (-1.2, 1.2, -1.2, 1.2), "cmap": 'jet'}
fig, ax = plt.subplots(1, 1, figsize=(10,7))

circle1 = plt.Circle((0,0),rs, color = 'black', fill = True)
circle2 = plt.Circle((0,0),rl, linewidth = 2,color = 'red', fill = False)

diff_x = x_range[1] - x_range[0]
diff_y = y_range[1] - y_range[0]
diff_z = z_range[1] - z_range[0]

rect = patches.Rectangle((x_range[0],y_range[0]),diff_x,diff_y,linewidth=2,edgecolor='r',facecolor='none')

electrons[s_y0 & s_x0].plot.hexbin(x='px', y='py', bins="log", ax=ax, **kwargs)
ax.add_artist(circle1)
ax.add_artist(circle2)

In [2]:
# Restricting sample to small 1x1 region chosen
# 'Region' is some x,z coordinate on a fld plot
# Can these 4 lines be made more concise?
df = df.loc[lambda df: df["x"] < region[0] + 0.5] 
df = df.loc[lambda df: df["x"] > region[0] - 0.5]
df = df.loc[lambda df: df["z"] < region[1] + 0.5] 
df = df.loc[lambda df: df["z"] > region[1] - 0.5]



# Chooses 5 particles that vary from lowest to highest energy within a sample 
def Particle_Select():
    df['eng_sqrd'] = (df['px']**2 + df['py']**2 + df['pz']**2)
    
    min = df.loc[lambda df: df['eng_sqrd'] == df["eng_sqrd"].min()]
    med = df.loc[lambda df: df['eng_sqrd'] == df["eng_sqrd"].median()]
    max = df.loc[lambda df: df['eng_sqrd'] == df["eng_sqrd"].max()]
    
    minmed = df.loc[lambda df: df['eng_sqrd'] < df["eng_sqrd"].median()]
    minmed = minmed.loc[lambda df: minmed['eng_sqrd'] == minmed["eng_sqrd"].median()]
    
    medmax = df.loc[lambda df: df['eng_sqrd'] > df["eng_sqrd"].median()]
    medmax = medmax.loc[lambda df: medmax['eng_sqrd'] == medmax["eng_sqrd"].median()]
    
    r_min = np.array([min["x"].median(),min["y"].median(),min["z"].median()])
    p_min = np.array([min["px"].median(),min["py"].median(),min["pz"].median()])
    
    r_minmed = np.array([minmed["x"].median(),minmed["y"].median(),minmed["z"].median()])
    p_minmed = np.array([minmed["px"].median(),minmed["py"].median(),minmed["pz"].median()])
    
    r_med = np.array([med["x"].median(),med["y"].median(),med["z"].median()])
    p_med = np.array([med["px"].median(),med["py"].median(),med["pz"].median()])
    
    r_medmax = np.array([medmax["x"].median(),medmax["y"].median(),medmax["z"].median()])
    p_medmax = np.array([medmax["px"].median(),medmax["py"].median(),medmax["pz"].median()])
    
    r_max = np.array([max["x"].median(),max["y"].median(),max["z"].median()])
    p_max = np.array([max["px"].median(),max["py"].median(),max["pz"].median()])

    return r_min, r_minmed, r_med, r_medmax, r_max, p_min, p_minmed, p_med, p_medmax, p_max, 


r0,r1,r2,r3,r4,p0,p1,p2,p3,p4 = Particle_Select()


-1